<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Create-or-Remove-Users" data-toc-modified-id="Create-or-Remove-Users-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Create or Remove Users</a></span></li><li><span><a href="#Add-Users-to-Groups" data-toc-modified-id="Add-Users-to-Groups-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Add Users to Groups</a></span></li><li><span><a href="#Modify-User-Password" data-toc-modified-id="Modify-User-Password-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Modify User Password</a></span></li><li><span><a href="#Create-Fundamental-Datasets" data-toc-modified-id="Create-Fundamental-Datasets-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create Fundamental Datasets</a></span></li></ul></div>

- Reference: https://tableau.github.io/server-client-python/docs/api-ref

In [1]:
import tableauserverclient as TSC
import pandas as pd
import numpy as np
import pymssql
from dateutil.relativedelta import relativedelta

pw_dm = open("D:/Github/Work/Tableau/datamart_password.txt", "r").readline()
pw_ts = open("D:/Github/Work/Tableau/tableau_server_password.txt", "r").readline()

# SIgn In
tableau_auth = TSC.TableauAuth("6363", pw_ts)
server = TSC.Server("https://datalab.hdc-dvp.com", use_server_version=True)

req_opts = TSC.RequestOptions(pagesize=1000)

In [2]:
conn = pymssql.connect(server="133.186.215.165", database="HDCMART", user="hdcmart_user", password=pw_dm, charset="UTF8")
query = f"""
SELECT id_sabun AS num, ds_hname AS name, ds_email AS email, ds_bonbu AS hq, ds_dept AS dept, ds_occupation AS job, DS_EMPTYPE AS sub, DS_TY_DEPT AS ty
FROM DATAMART_DAAV_BASEINFO_DETAIL_ALL
WHERE cd_corp = 'A101';
"""
hr = pd.read_sql(query, conn)
hr.columns = hr.columns.str.lower()
for col in hr.columns:
    if pd.api.types.is_string_dtype(hr[col]):
        hr[col] = hr[col].str.encode("latin-1", errors="ignore").str.decode("euc-kr", errors="ignore")
hr.loc[hr["num"]=="1", "num"] = "0001"

hr_dic = {str(row["num"]):(row["hq"], row["dept"], row["job"], row["sub"], row["name"]) for _, row in hr.iterrows()}

# all_groups.csv
with server.auth.sign_in(tableau_auth):
    all_groups, _ = server.groups.get(req_options=req_opts)
all_groups = [(group.name, group.id) for group in all_groups]
all_groups = pd.DataFrame(all_groups, columns=["group", "group_id"])
# all_groups.to_csv("./all_groups.csv", encoding="euc-kr")

In [4]:
# all_users_df.csv
with server.auth.sign_in(tableau_auth):
    all_users, _ = server.users.get(req_options=req_opts)
all_users_df = [(user.id, user.name, user.fullname, user.site_role) + hr_dic[user.name] if user.name in hr_dic.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None) for user in all_users]
all_users_df = pd.DataFrame(all_users_df, columns=["user_id", "user_name", "user_fnm", "site_role", "hq", "dept", "job", "sub", "name"])
all_users_df = all_users_df.sort_values(["user_fnm"])
all_users_df.to_csv("./all_users_df.csv", encoding="euc-kr")

# Create or Remove Users

In [ ]:
def create_or_remove_users(users_to_create):
    with server.auth.sign_in(tableau_auth):
        all_users, _ = server.users.get(req_options=req_opts)
        for _, row in hr.iterrows():
            if row["num"] in users_to_create["num"].tolist() + ["0001", "6415", "4686", "Test", "goodcen", "creator"]:
                if row["num"] not in [i.name for i in all_users]:
                    # Decide site role for each user.
                    if row["num"] in ["4100435"]: # 이재호
                        user = TSC.UserItem(row["num"], "SiteAdministratorCreator")
                    elif row["num"] in ["4001380"]: # 김아현
                        user = TSC.UserItem(row["num"], "Creator")
                    else:
                        user = TSC.UserItem(row["num"], "Viewer")

                    # Add a user
                    user = server.users.add(user)
                    print(f"Created a user; {row['name']}({row['dept']})({row['job']})")
                else:
                    # Update user information
                    user = [i for i in all_users if i.name == row["num"]][0]

                if row["job"] in ["회장", "부회장", "대표이사", "CSO", "본부장", "팀장", "실장", "그룹장", "부문장", "단장", "PL", "PM", "TFT장", "건축소장", "토목소장"]:
                    tar_fullname = f"{row['name']}({row['dept']})({row['job']})"
                else:
                    tar_fullname = f"{row['name']}({row['dept']})"
                if user.fullname != tar_fullname:
                    print(f"Updated user information; {user.fullname} -> {tar_fullname}")
                    user.fullname = tar_fullname
                user.email = row["email"]
                user = server.users.update(user)
            elif row["num"] in [i.name for i in all_users]:
                user = [i for i in all_users if i.name == row["num"]][0]
                server.users.remove(user.id)
                print(f"Removed a user; {user.fullname}")

    print("All completed!")
    
users_to_create = hr[(hr["num"].isin(["4001386", "6084", "4395", "6111", "6294", "4001360", "6161", "1101320", "4001401"]))]
create_or_remove_users(users_to_create)

In [3]:
def create_or_remove_users(users_to_create):
    with server.auth.sign_in(tableau_auth):
        all_users, _ = server.users.get(req_options=req_opts)
        for _, row in hr.iterrows():
            if row["num"] in users_to_create["num"].tolist() + ["0001", "6415", "4686", "Test", "goodcen", "creator"]:
                if row["num"] not in [i.name for i in all_users]:
                    # Decide site role for each user.
                    if row["num"] in ["4100435"]: # 이재호
                        user = TSC.UserItem(row["num"], "SiteAdministratorCreator")
                    elif row["num"] in ["4001380"]: # 김아현
                        user = TSC.UserItem(row["num"], "Creator")
                    else:
                        user = TSC.UserItem(row["num"], "Viewer")

                    # Add a user
                    user = server.users.add(user)
                    print(f"Created a user; {row['name']}({row['dept']})({row['job']})")
                else:
                    # Update user information
                    user = [i for i in all_users if i.name == row["num"]][0]

                if row["job"] in ["회장", "부회장", "대표이사", "CSO", "본부장", "팀장", "실장", "그룹장", "부문장", "단장", "PL", "PM", "TFT장", "건축소장", "토목소장"]:
                    tar_fullname = f"{row['name']}({row['dept']})({row['job']})"
                else:
                    tar_fullname = f"{row['name']}({row['dept']})"
                if user.fullname != tar_fullname:
                    print(f"Updated user information; {user.fullname} -> {tar_fullname}")
                    user.fullname = tar_fullname
                user.email = row["email"]
                user = server.users.update(user)
            elif row["num"] in [i.name for i in all_users]:
                user = [i for i in all_users if i.name == row["num"]][0]
                server.users.remove(user.id)
                print(f"Removed a user; {user.fullname}")

    print("All completed!")
    
users_to_create = hr[(hr["job"]=="회장") | (hr["job"]=="부회장") | (hr["job"]=="대표이사") | (hr["job"]=="CSO") | (hr["job"]=="본부장") | ((hr["sub"]!="정보기술/CS외주") & (hr["job"]=="팀장")) | (hr["job"]=="실장") | (hr["job"]=="그룹장") | (hr["job"]=="부문장") | (hr["job"]=="단장") | (hr["job"]=="PL") | (hr["job"]=="PM") | (hr["job"]=="TFT장") | (hr["job"]=="건축소장") | (hr["job"]=="토목소장") | ((hr["dept"]=="경영분석팀") & (hr["sub"]=="일반직") & (hr["job"]=="팀원") & (~hr["num"].isin(["4113"]))) | ((hr["dept"]=="HR혁신팀") & (hr["sub"]=="일반직") & (hr["job"]=="팀원")) | (hr["dept"]=="상근고문") | ((hr["dept"]=="안전기획팀") & (hr["sub"]!="파견직")) | (hr["num"].isin(["6363", "5899", "5947", "4100435", "4001380", "4100222", "1101320", "4001360", "4001386", "1101291", "6294", "6111", "6048", "6403", "6032", "6354", "5704", "5720", "5707", "6084", "6161", "5934", "5218", "6110", "4632", "5104", "5892", "6335", "6353", "6284", "5467", "6281", "6427", "6012", "6360", "6122", "5994", "6385", "6310", "6389", "5516", "6455", "5903", "5919", "6457", "6036", "5308", "6469", "6034", "6333", "5634", "3991", "4154", "4826", "5381", "5477", "5555", "5634", "5673", "5720", "5892", "5969", "6033", "6253", "6376", "6414", "6453", "6456", "6460", "6045", "6448", "3684", "3805", "3872", "3911", "3991", "4009", "4057", "4078", "4540", "4598", "6446", "6186"]))]
create_or_remove_users(users_to_create)

All completed!


# Add Users to Groups

In [4]:
# all_users_df.csv
with server.auth.sign_in(tableau_auth):
    all_users, _ = server.users.get(req_options=req_opts)
all_users_df = [(user.id, user.name, user.fullname, user.site_role) + hr_dic[user.name] if user.name in hr_dic.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None) for user in all_users]
all_users_df = pd.DataFrame(all_users_df, columns=["user_id", "user_name", "user_fnm", "site_role", "hq", "dept", "job", "sub", "name"])
all_users_df = all_users_df.sort_values(["user_fnm"])
all_users_df.to_csv("./all_users_df.csv", encoding="euc-kr")

def add_to_or_remove_from_groups(tar_users, tar_grp, tar_grp_text):
    with server.auth.sign_in(tableau_auth):
        groups, pagination_item = server.groups.get(req_options=req_opts)
        tar_grp = [i for i in groups if i.id == tar_grp][0]
        print(f"{tar_grp.name}:")
        for _, row in all_users_df.iterrows():
            user_id = row["user_id"]
            if user_id in tar_users["user_id"].tolist():
                try:
                    server.groups.add_user(tar_grp, user_id)
                    print(f"\tAdded {row['user_fnm']}")
                except:
                    continue
            else:
                try:
                    server.groups.remove_user(tar_grp, user_id)
                    print(f"\tRemoved {row['user_fnm']}")
                except:
                    continue
# 관리자
tar_users = all_users_df[(all_users_df["user_name"].isin(["6363", "5899", "5947"])) | ((all_users_df["job"]=="팀장") & (all_users_df["dept"]=="디지털플랫폼팀"))]
tar_grp = "6e2fc4f0-46f2-4114-8b25-341b38af9f7d"
add_to_or_remove_from_groups(tar_users, tar_grp, "관리자")

# 회장/부회장/대표이사/본부장
tar_users = all_users_df[(all_users_df["job"]=="회장") | (all_users_df["job"]=="부회장") | (all_users_df["job"]=="대표이사") | (all_users_df["job"]=="CSO") | (all_users_df["job"]=="본부장") | ((all_users_df["job"]=="팀장") & (all_users_df["dept"]=="디지털플랫폼팀")) | (all_users_df["dept"]=="상근고문") | (all_users_df["user_name"].isin(["6403", "1101291", "5634"])) | (all_users_df["user_name"]=="goodcen")]
tar_grp = "f827e0fe-ff32-4443-9c40-701c94a13ba1"
add_to_or_remove_from_groups(tar_users, tar_grp, "회장/부회장/대표이사/본부장")

# 팀장/실장/그룹장/부문장/단장/PL/PM/TFT장
tar_users = all_users_df[((all_users_df["sub"] != "정보기술/CS외주") & (all_users_df["job"]=="팀장")) | (all_users_df["job"]=="실장") | (all_users_df["job"]=="그룹장") | (all_users_df["job"]=="부문장") | (all_users_df["job"]=="단장") | (all_users_df["job"]=="PL") | (all_users_df["job"]=="PM") | (all_users_df["job"]=="TFT장")]
tar_grp = "39dc2d08-d291-4534-97d5-b1a6b3b6f67c"
add_to_or_remove_from_groups(tar_users, tar_grp, "팀장/실장/그룹장/부문장/단장/PL/PM/TFT장")

# 건축소장/토목소장
tar_users = all_users_df[(all_users_df["job"]=="건축소장") | (all_users_df["job"]=="토목소장")]
tar_grp = "05fa4bd0-e1df-4971-bb89-4e39f24782d7"
add_to_or_remove_from_groups(tar_users, tar_grp, "건축소장/토목소장")

# HR혁신팀
tar_users = all_users_df[(all_users_df["user_name"].isin(["4100222"])) | ((all_users_df["dept"]=="HR혁신팀") & (all_users_df["sub"]=="일반직"))]
tar_grp = "1ac1f9c5-d4b9-47d9-8c40-618df8f87f6b"
add_to_or_remove_from_groups(tar_users, tar_grp, "HR혁신팀")

# 경영현안 회의
tar_users = all_users_df[(all_users_df["user_name"].isin(["3991", "4154", "4826", "5381", "5477", "5555", "5634", "5673", "5720", "5892", "5969", "6033", "6253", "6376", "6414", "6453", "6456", "6460", "3684", "3805", "3872", "3911", "3991", "4009", "4057", "4078", "4540", "4598", "6446", "6415", "4686", "5704", "6045", "6012", "4001380", "6036", "6442", "6248", "4095", "5888"]))]
tar_grp = "388136df-5404-4b2d-8805-0276a6c3080a"
add_to_or_remove_from_groups(tar_users, tar_grp, "경영현안 회의")
            
# 본부장 회의 일반 사용자
tar_users = all_users_df[(all_users_df["user_name"].isin(["6427", "6457", "6036", "6335", "4001380", "4100435", "6281", "4001341", "4001360", "4001386", "5653", "6012", "6360", "6122", "5994", "6385", "6389", "5516", "6455", "5835", "5903", "5919", "6354", "5704", "5720", "4632", "5308", "5888", "6428", "6034", "6333", "5467", "6281", "6186", "6337"]))]
tar_grp = "0ef096dd-2c99-4473-b80b-855d1c0b29e1"
add_to_or_remove_from_groups(tar_users, tar_grp, "본부장 회의 일반 사용자")
            
# 경영 현황 일반 사용자
tar_users = all_users_df[((all_users_df["dept"]=="경영분석팀") & (all_users_df["sub"]=="일반직") & (all_users_df["job"]=="팀원") & (~all_users_df["user_name"].isin(["4113"]))) | (all_users_df["user_name"].isin(["6360", "6122", "5994", "6385", "6310", "6389", "5516", "6455", "5835", "5903", "5919", "6354", "5704", "5720", "6469", "5308", "6034", "6333", "5467", "6281"]))]
tar_grp = "91d8104c-e669-4ea2-a5fa-27b6be938858"
add_to_or_remove_from_groups(tar_users, tar_grp, "경영 현황 일반 사용자")

# HR 현황 일반 사용자
tar_users = all_users_df[((all_users_df["dept"]=="안전기획팀") & (all_users_df["sub"]!="파견직")) | (all_users_df["user_name"].isin(["6385", "6385", "6122", "6455", "5903", "5919", "6354", "5704", "5720", "6284", "5104", "6469", "5308", "6034", "6333", "5467", "6281"]))]
tar_grp = "fef23089-cfd7-403d-8982-894afac13b32"
add_to_or_remove_from_groups(tar_users, tar_grp, "HR 현황 일반 사용자")
            
# 영업 현황 일반 사용자
tar_users = all_users_df[((all_users_df["dept"]=="경영분석팀") & (all_users_df["sub"]=="일반직") & (all_users_df["job"]=="팀원") & (~all_users_df["user_name"].isin(["4113"]))) | (all_users_df["user_name"].isin(["5218", "6337", "4632", "6110", "6122", "5994", "6385", "6310", "6389", "5516", "6455", "5835", "5903", "5919", "6354", "5704", "5720", "6284", "5934", "5104", "6469", "6457", "6036", "5308", "6034", "6333", "5467", "6281"]))]
tar_grp = "0715f81d-9385-4fc8-8c2a-76f8b9481dac"
add_to_or_remove_from_groups(tar_users, tar_grp, "영업 현황 일반 사용자")

# 현장 현황 일반 사용자
tar_users = all_users_df[((all_users_df["dept"]=="경영분석팀") & (all_users_df["sub"]=="일반직") & (all_users_df["job"]=="팀원") & (~all_users_df["user_name"].isin(["4113"]))) | ((all_users_df["dept"]=="안전기획팀") & (all_users_df["sub"]!="파견직")) | (all_users_df["user_name"].isin(["5947", "6353", "6360", "6122", "5994", "6385", "6310", "6389", "5516", "6455", "5835", "5903", "5919", "6354", "5704", "5720", "6284", "5934", "5104", "6469", "5308",
"6034", "6333", "5467", "6281"]))]
tar_grp = "fcace1dd-cfa8-4f64-8b20-803c739e1e3c"
add_to_or_remove_from_groups(tar_users, tar_grp, "현장 현황 일반 사용자")
            
# 직무정보 일반 사용자
tar_users = all_users_df[(all_users_df["user_name"].isin(["5892"]))]
tar_grp = "b2e29db6-c54f-45ce-b021-31596d75efc1"
add_to_or_remove_from_groups(tar_users, tar_grp, "직무정보 일반 사용자")
            
# 안전 현황 일반 사용자
tar_users = all_users_df[((all_users_df["hq"].isin(["건설본부", "CSO직속"])) & (all_users_df["sub"]!="파견직")) | (all_users_df["dept"].isin(["미래전략팀", "디지털플랫폼팀"]) & (all_users_df["sub"]!="파견직")) | (all_users_df["user_name"].isin(["5467", "6281"]))]
tar_grp = "bfbfae75-a700-4a3f-aba1-7d8bb7f81663"
add_to_or_remove_from_groups(tar_users, tar_grp, "안전 현황 일반 사용자")
            
print("All completed!")

관리자:
회장/부회장/대표이사/본부장:
팀장/실장/그룹장/부문장/단장/PL/PM/TFT장:
건축소장/토목소장:
HR혁신팀:
경영현안 회의:
본부장 회의 일반 사용자:
경영 현황 일반 사용자:
HR 현황 일반 사용자:
영업 현황 일반 사용자:
현장 현황 일반 사용자:
직무정보 일반 사용자:
안전 현황 일반 사용자:
All completed!


# Modify User Password

In [12]:
def modify_user_password(nums):
    with server.auth.sign_in(tableau_auth):
        all_users, _ = server.users.get(req_options=req_opts)
        for user in all_users:
            if user.name in nums:
                user = server.users.update(user, password="secretpassword")
    print("All Completed!")
    
nums = ["1101291"]
modify_user_password(nums)

All Completed!


# Create Fundamental Datasets

In [11]:
# user_group.csv
user_group = list()
with server.auth.sign_in(tableau_auth):
    users, pagination_item = server.users.get(req_options=req_opts)
    for user in users:
        pagination_item = server.users.populate_groups(user, req_options=req_opts)
        user_group.extend([(user.id, user.name, user.fullname, user.site_role) + hr_dic[user.name] + (group.name,) if user.name in hr_dic.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None, group.name) for group in user.groups if group.name != "All Users"])
user_group = pd.DataFrame(user_group, columns=["user_id", "user_name", "user_fnm", "site_role", "hq", "dept", "job", "name", "group"])

user_group.to_csv("user_group.csv", index=False, encoding="euc-kr")

# group_user.csv
group_user = list()
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    for group in groups[1:]:
        pagination_item = server.groups.populate_users(group, req_options=req_opts)
        group_user.extend([(group.name,) + hr_dic[user.name] + (user.site_role,) if user.name in hr_dic.keys() else (group.name, None, None, None, None, user.name, user.site_role) for user in group.users])

group_user = pd.DataFrame(group_user, columns=["group", "hq", "dept", "job", "sub", "name", "site_role"])

group_user.to_csv("group_user.csv", index=False, encoding="euc-kr")

# user_workbook
user_wb = list()
with server.auth.sign_in(tableau_auth):
    users, pagination_item = server.users.get(req_options=req_opts)
    for user in users:
        pagination_item = server.users.populate_workbooks(user, req_options=req_opts)
        user_wb.extend([(user.id, user.name, user.fullname, user.site_role) + hr_dic[user.name] + (wb.name,) if user.name in hr_dic.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None, wb.name) for wb in user.workbooks if wb.project_name=="EIS"])
user_wb = pd.DataFrame(user_wb, columns=["user_id", "user_name", "user_fnm", "site_role", "hq", "dept", "job", "name", "wb"])

user_wb.to_csv("user_wb.csv", index=False, encoding="euc-kr")

# have_account
have_account = hr[["num", "hq", "dept", "name", "sub", "job", "rank", "email"]]
have_account["have"] = have_account["num"].apply(lambda x:True if str(x) in all_users["user_name"].tolist() else False)

have_account.to_csv("D:/Github/Work/Tableau/have_account.csv", index=False, encoding="euc-kr")

print("All completed!")